In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
sns.set(palette='colorblind')
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline 
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import cross_val_predict, RandomizedSearchCV, GridSearchCV

In [2]:
raw_data = pd.read_csv('titanic.csv')
raw_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
# Checkpoint
data = raw_data.copy()
# Dropping Name and Cabin
data.drop(['Name', 'Cabin'], axis=1, inplace=True)
# Column to change list
column_to_change = ['Pclass', 'SibSp', 'Parch']
data[column_to_change] = data[column_to_change].apply(lambda x: x.astype('category'))
# Instantiate LabelEncoder
lb_encode = LabelEncoder()
data['Ticket'] = lb_encode.fit_transform(data['Ticket'])
# Set Passenger_id as index
data = data.set_index('PassengerId')
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,
1,0,3,male,22.0,1,0,523,7.2500,S
2,1,1,female,38.0,1,0,596,71.2833,C
3,1,3,female,26.0,0,0,669,7.9250,S
4,1,1,female,35.0,1,0,49,53.1000,S
5,0,3,male,35.0,0,0,472,8.0500,S


In [5]:
def validation_fun(df):
    df.drop(['Name', 'Cabin'], axis=1, inplace=True)
    # Column to change list
    column_to_change = ['Pclass', 'SibSp', 'Parch']
    df[column_to_change] = df[column_to_change].apply(lambda x: x.astype('category'))
    # Instantiate LabelEncoder
    lb_encode = LabelEncoder()
    df['Ticket'] = lb_encode.fit_transform(df['Ticket'])
    # Set Passenger_id as index
    df = df.set_index('PassengerId')
    return df

In [6]:
# Split data into inputs and targets
targets = data['Survived']
inputs = data.drop('Survived', axis=1)

In [7]:
cat_names = inputs.select_dtypes(exclude='number').columns
num_names = inputs.select_dtypes(include='number').columns

cat_pipeline = Pipeline(
                        [
                            ('cat_imputer', SimpleImputer(strategy='most_frequent')),
                            ('encoder', OneHotEncoder())
                        ]
)
num_pipeline = Pipeline(
                        [
                            ('num_imputer', SimpleImputer(strategy='median')),
                            ('scaler', StandardScaler())
                        ]
)
full_pipeline = ColumnTransformer(
                                  [
                                      ('cat_pipeline', cat_pipeline, cat_names),
                                      ('num_pipeline', num_pipeline, num_names)
                                  ]
)
pca = PCA()
inputs_scaled = full_pipeline.fit_transform(inputs)

In [8]:
# Instantiate LogisticRegression
lg_reg = LogisticRegression()
lg_reg.fit(inputs_scaled, targets)
baseline_pred = lg_reg.predict(inputs_scaled)
baseline_accuracy = accuracy_score(targets, baseline_pred)
baseline_precision = precision_score(targets, baseline_pred)
baseline_recall = recall_score(targets, baseline_pred)
print(f'The baseline accuracy is {round(baseline_accuracy, 2)*100}%')
print(f'The baseline precision is {round(baseline_precision, 2)*100}%')
print(f'The baseline recall is {round(baseline_recall, 2)*100}%')

The baseline accuracy is 82.0%
The baseline precision is 79.0%
The baseline recall is 72.0%


In [9]:
# Instantiate XGBClassifier
xgb_class = xgb.XGBClassifier(n_estimators=50, seed=42, objective='reg:logistic')
xgb_class.fit(inputs_scaled, targets)
xgb_pred = xgb_class.predict(inputs_scaled)
xgb_accuracy = accuracy_score(targets, xgb_pred)
xgb_precision = precision_score(targets, xgb_pred)
xgb_recall = recall_score(targets, xgb_pred)
print(f'The XGBClassifier accuracy is {round(xgb_accuracy, 2)*100}%')
print(f'The XGBClassifier precision is {round(xgb_precision, 2)*100}%')
print(f'The XGBClassifier recall is {round(xgb_recall, 2)*100}%')

The XGBClassifier accuracy is 97.0%
The XGBClassifier precision is 98.0%
The XGBClassifier recall is 94.0%


In [10]:
# Instantiate RandomForestClassifier
rf_class = RandomForestClassifier(n_estimators=200, random_state=42)
rf_class.fit(inputs_scaled, targets)
rf_pred = rf_class.predict(inputs_scaled)
rf_accuracy = accuracy_score(targets, rf_pred)
rf_precision = precision_score(targets, rf_pred)
rf_recall = recall_score(targets, rf_pred)
print(f'The RandomForestClassifier accuracy is {round(rf_accuracy, 2)*100}%')
print(f'The RandomForestClassifier precision is {round(rf_precision, 2)*100}%')
print(f'The RandomForestClassifier recall is {round(rf_recall, 2)*100}%')

The RandomForestClassifier accuracy is 100.0%
The RandomForestClassifier precision is 100.0%
The RandomForestClassifier recall is 99.0%


In [11]:
# Instantiate GradientBoostingClassifier
gb_class = GradientBoostingClassifier(n_estimators=200, random_state=42)
gb_class.fit(inputs_scaled, targets)
gb_pred = gb_class.predict(inputs_scaled)
gb_accuracy = accuracy_score(targets, gb_pred)
gb_precision = precision_score(targets, gb_pred)
gb_recall = recall_score(targets, gb_pred)
print(f'The GradientBoostingClassifier accuracy is {round(gb_accuracy, 2)*100}%')
print(f'The GradientBoostingClassifier precision is {round(gb_precision, 2)*100}%')
print(f'The GradientBoostingClassifier recall is {round(gb_recall, 2)*100}%')

The GradientBoostingClassifier accuracy is 93.0%
The GradientBoostingClassifier precision is 94.0%
The GradientBoostingClassifier recall is 88.0%


In [12]:
# Hyper parameter tunning of GradientBoostingClassifier
params = {
    "n_estimators":[5,50,250,500],
    "max_depth":[1,3,5,7,9],
    "learning_rate":[0.01,0.1,1,10,100]
}
gb_cv = RandomizedSearchCV(estimator=gb_class, param_distributions=params,
                               scoring='accuracy', cv=5)
gb_cv.fit(inputs_scaled, targets)
print(f'The best accuracy of GradientBoostingClassifier is {gb_cv.best_score_}')

The best accuracy of GradientBoostingClassifier is 0.8170861841692298


In [13]:
params = {
    "n_estimators":[64, 128, 256],
    "max_depth":[2, 8, 16]
}
rf_cv = GridSearchCV(estimator=rf_class, param_grid=params,
                               scoring='accuracy', cv=5)
rf_cv.fit(inputs_scaled, targets)
print(f'The best accuracy of RandomForestClassifier is {rf_cv.best_score_}')

The best accuracy of RandomForestClassifier is 0.8316740945326722


In [14]:
# Hyper-parameter tunning of XGBClassifier
params = {
    "learning_rate": np.arange(0.05, 1.05, 0.05),
    "subsample": np.arange(0.05, 1.05, 0.05),
    "n_estimators": [100,200,300,400,500]
}
xgb_cv = RandomizedSearchCV(estimator=xgb_class, param_distributions=params, n_iter=25,
                               scoring='accuracy', cv=5, verbose=1)
xgb_cv.fit(inputs_scaled, targets)
print(f'The best accuracy of XGBClassifier is {xgb_cv.best_score_}')

Fitting 5 folds for each of 25 candidates, totalling 125 fits
The best accuracy of XGBClassifier is 0.829458288870755


In [15]:
cv_pred = cross_val_predict(xgb_cv.best_estimator_, inputs_scaled, targets)
cv_accuracy = accuracy_score(targets, cv_pred)
cv_precision = precision_score(targets, cv_pred)
cv_recall = recall_score(targets, cv_pred)
print(f'The Evaluation accuracy is {round(cv_accuracy, 2)*100}%')
print(f'The Evaluation precision is {round(cv_precision, 2)*100}%')
print(f'The Evaluation recall is {round(cv_recall, 2)*100}%')

The Evaluation accuracy is 83.0%
The Evaluation precision is 80.0%
The Evaluation recall is 74.0%


In [16]:
# Load the test Data 
test_set = pd.read_csv('titanic.csv')
test_set['Parch'] = np.where(test_set['Parch']==9, np.nan, test_set['Parch'])
test_set_data = validation_fun(test_set)
test_scaled = full_pipeline.transform(test_set_data)

In [17]:
best_model = xgb_cv.best_estimator_

In [18]:
test_pred = xgb_class.predict(test_scaled)
submission_df = pd.DataFrame()
submission_df['PassengerId'] = test_set_data.index
submission_df['Survived'] = test_pred
submission_df.set_index('PassengerId', inplace=True)
submission_df.head()

,Survived
PassengerId,
1,0
2,1
3,1
4,1
5,0


In [19]:
submission_df.to_csv('submission.csv')